# Strategy Rules

- Determine Upward Trend: Identify an upward trend by checking if the closing price is above a moving average (e.g., EMA).
- Check for Bearish Volatility: Ensure there is no bearish volatility by analyzing the ATR indicators.
- Entry Signal: Generate an entry signal if there is an upward trend and no bearish volatility. 
- Exit Signal: Implement a trailing stop-loss to generate an exit signal and protect profits.
- tight trailing stop when if intrade and bearish volatility.


# to do 
- display candle stick chart 
- create indicator on webpage and display with all calculations 
- add custom indiactor for vbt 
- data allignment explanantion 
- calculate possible outcomes for strategy 
- display outcomes 
- go to code for deployment and pu on website 

In [1]:
import pandas as pd
import numpy as np
import vectorbt as vbt
from datetime import datetime
import os 
from numba import njit
from vectorbt.portfolio.enums import Direction, SizeType
import talib
import requests
import time


symbol = "BTC-USDT"
timeframe = "1day"
market_type = "spot"
start_time = "2021-01-01 00:00:00"
end_time = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")
# end_time = "2021-06-10 00:00:00"

skip_fetch = False


/Users/andre/Documents/Python_local/trading_clone_singapore_DO/trading_singapore_digitalocean/pandas/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:

if not skip_fetch:
    print("Fetching data...")
    def fetch_kucoin_candles_chunk(symbol, market_type="spot", timeframe="1min", start_time=None, end_time=None):
        time.sleep(0.2)  # Rate limiting
        base_url = "https://api.kucoin.com" if market_type.lower() == "spot" else "https://api-futures.kucoin.com"
        url = base_url + "/api/v1/market/candles"
        params = {"type": timeframe, "symbol": symbol.upper()}
        if start_time:
            params["startAt"] = int(time.mktime(time.strptime(start_time, "%Y-%m-%d %H:%M:%S")))
        if end_time:
            params["endAt"] = int(time.mktime(time.strptime(end_time, "%Y-%m-%d %H:%M:%S")))
        try:
            resp = requests.get(url, params=params, timeout=10)
            resp.raise_for_status()
            data = resp.json()
            if data.get("code") == "200000":
                return data["data"]
            else:
                raise Exception(f"KuCoin API error: {data}")
        except requests.exceptions.RequestException as e:
            raise Exception(f"Request failed: {str(e)}")

    def fetch_all_kucoin_candles(symbol, market_type="spot", timeframe="1min", start_time=None, end_time=None):
        chunks = []
        current_end = end_time
        start_timestamp = int(time.mktime(time.strptime(start_time, "%Y-%m-%d %H:%M:%S")))
        
        while True:
            try:
                chunk = fetch_kucoin_candles_chunk(symbol, market_type, timeframe, start_time, current_end)
                if not chunk:
                    print("No more data available")
                    break
                earliest_ts = int(chunk[-1][0])
                print(f"Fetched {len(chunk)} candles from {datetime.fromtimestamp(earliest_ts)}")
                chunks.extend(chunk)
                if earliest_ts <= start_timestamp:
                    print("Reached start time")
                    break
                current_end = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime(earliest_ts - 60))
            except Exception as e:
                print(f"Error occurred: {str(e)}")
                break
        
        if not chunks:
            return []
        
        chunks.sort(key=lambda x: x[0])  # Sort by timestamp
        result = [candle for candle in chunks if start_timestamp <= int(candle[0]) <= int(time.mktime(time.strptime(end_time, "%Y-%m-%d %H:%M:%S")))]
        print(f"Total candles fetched: {len(result)}")
        return result


    data = fetch_all_kucoin_candles(symbol, market_type, timeframe, start_time, end_time)
    print(f"Final data length: {len(data)}")
    data
else:
    print("Skipping fetch...")

Fetching data...
Fetched 1467 candles from 2021-01-01 00:00:00
Reached start time
Total candles fetched: 1467
Final data length: 1467


In [3]:

# Assuming 'data' is the list of candlestick data
df = pd.DataFrame(data)
df[['timestamp', 'open', 'close', 'high', 'low']] = df[[0, 1, 2, 3, 4]]
df.drop([0, 1, 2, 3, 4, 5, 6], axis=1, inplace=True)

# Explicitly cast 'timestamp' to numeric type
df['timestamp'] = pd.to_numeric(df['timestamp'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s', utc=True)

df.set_index('timestamp', inplace=True)
df.sort_index(inplace=True)
df[['open', 'close', 'high', 'low']] = df[['open', 'close', 'high', 'low']].astype(float)
df.info()
df

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1467 entries, 2021-01-01 00:00:00+00:00 to 2025-01-06 00:00:00+00:00
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    1467 non-null   float64
 1   close   1467 non-null   float64
 2   high    1467 non-null   float64
 3   low     1467 non-null   float64
dtypes: float64(4)
memory usage: 57.3 KB


,open,close,high,low
timestamp,,,,
2021-01-01 00:00:00+00:00,28924.9,29333.2,29607.1,28633.0
2021-01-02 00:00:00+00:00,29333.2,32185.4,33286.8,28949.2
2021-01-03 00:00:00+00:00,32181.3,33000.4,34766.0,31500.0
2021-01-04 00:00:00+00:00,33005.9,31993.0,33600.2,27500.0
2021-01-05 00:00:00+00:00,31993.1,33938.6,34371.0,29806.1
...,...,...,...,...
2025-01-02 00:00:00+00:00,94584.2,96983.8,97839.1,94380.5
2025-01-03 00:00:00+00:00,96983.9,98173.9,98976.9,96105.4
2025-01-04 00:00:00+00:00,98173.9,98215.4,98782.5,97537.4


In [4]:
import plotly.graph_objects as go
fig = go.Figure()

fig.add_trace(go.Candlestick(x=df.index,
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close']))

fig.update_layout(
    title='BTC-USDT Candlestick 1day',
    xaxis_title='Time',
    yaxis_title='Price',
    template='plotly_dark',
    hovermode='x unified',

    
    xaxis_rangeslider_visible=False
)

fig.show()

## creating indicator to set signals

In [6]:
import pandas as pd
import numpy as np
import vectorbt as vbt

skip_cell = False

if not skip_cell:
    print("This cell is executed.")

    def momentum_strategy(close, high, low, atr_length, ema_length, ema2_length, vola_multiplier, vol_window):
        # Calculate EMA
        EMA = vbt.IndicatorFactory.from_talib('EMA')
        ema = EMA.run(close, timeperiod=ema_length).real.to_numpy()
        ema2 = EMA.run(close, timeperiod=ema2_length).real.to_numpy()

        # Calculate ATR
        ATR = vbt.IndicatorFactory.from_talib('ATR')
        atr = ATR.run(high, low, close, timeperiod=atr_length).real.to_numpy()

        # Function to calculate rolling max
        def rolling_max(arr, window):
            result = np.full_like(arr, np.nan)
            for i in range(window - 1, len(arr)):
                result[i] = np.max(arr[i - window + 1:i + 1])
            return result

        # Function to compute bearish volatility
        def compute_is_bearish_vol(high, low, atr, window):
            rm = rolling_max(high, window=window)
            return (rm - low) > (atr * vola_multiplier)

        # Calculate conditions
        is_bullish = (close > ema) & (close > ema2)
        is_bearish_vol = compute_is_bearish_vol(high, low, atr, window=vol_window)
        is_caution = is_bullish & (is_bearish_vol | (close < ema))
        signal_buy = is_bullish & ~is_caution

        # Define conditions and corresponding values
        conditions = [signal_buy, is_caution]
        values = [10, 5]

        # Create final signal using np.select
        signal = np.select(conditions, values, default=0)
        return signal.reshape(close.shape)

    # Create the indicator factory
    momentum_indicator = vbt.IndicatorFactory(
        class_name='MomentumStrategy',
        short_name='momentum',
        input_names=['Close', 'High', 'Low'],
        param_names=['atr_length', 'ema_length', 'ema2_length', 'vola_multiplier', 'vol_window'],
        output_names=['signal']
    ).from_apply_func(momentum_strategy)

    # Define parameter ranges
    atr_length = np.arange(5, 8, 1)  # Range from 5 to 7 with step 1
    ema_length = np.arange(100, 241, 20)  # Range from 100 to 240 with step 20
    ema2_length = np.arange(10, 50, 5)  # Range from 10 to 45 with step 5
    vola_multiplier = [1.6, 1.7, 1.8]  # Specific values for volatility multiplier
    vol_window = np.arange(7, 10, 1)  # Range from 7 to 9 with step 1

    # Run the indicator
    signal = momentum_indicator.run(
        df['close'], df['high'], df['low'],
        atr_length=atr_length,
        ema_length=ema_length,
        ema2_length=ema2_length,
        vola_multiplier=vola_multiplier,
        vol_window=vol_window,
        param_product=True
    )

    # Print the shape of the signal and store it in a DataFrame
    print(signal.signal.shape)
    df_indicator = signal.signal

else:
    print("This cell is skipped.")

This cell is executed.
(1467, 1728)


In [7]:
df_indicator

momentum_atr_length         5                                      ...   7  \
momentum_ema_length       100                                      ... 240   
momentum_ema2_length       10                                  15  ...  40   
momentum_vola_multiplier  1.6         1.7         1.8         1.6  ... 1.8   
momentum_vol_window         7   8   9   7   8   9   7   8   9   7  ...   9   
timestamp                                                          ...       
2021-01-01 00:00:00+00:00   0   0   0   0   0   0   0   0   0   0  ...   0   
2021-01-02 00:00:00+00:00   0   0   0   0   0   0   0   0   0   0  ...   0   
2021-01-03 00:00:00+00:00   0   0   0   0   0   0   0   0   0   0  ...   0   
2021-01-04 00:00:00+00:00   0   0   0   0   0   0   0   0   0   0  ...   0   
2021-01-05 00:00:00+00:00   0   0   0   0   0   0   0   0   0   0  ...   0   
...                        ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ...  ..   
2025-01-02 00:00:00+00:00  10   5   5  10  10  10  10  10  10  10  ...  10   
2025-01-03 00:00:00+00:00  10  10  10  10  10  10  10  10  10  10  ...  10   
2025-01-04 00:00:00+00:00  10  10  10  10  10  10  10  10  10  10  ...  10   
2025-01-05 00:00:00+00:00  10  10  10  10  10  10  10  10  10  10  ...  10   
2025-01-06 00:00:00+00:00  10  10  10  10  10  10  10  10  10  10  ...  10   

momentum_atr_length                                            
momentum_ema_length                                            
momentum_ema2_length       45                                  
momentum_vola_multiplier  1.6         1.7         1.8          
momentum_vol_window         7   8   9   7   8   9   7   8   9  
timestamp                                                      
2021-01-01 00:00:00+00:00   0   0   0   0   0   0   0   0   0  
2021-01-02 00:00:00+00:00   0   0   0   0   0   0   0   0   0  
2021-01-03 00:00:00+00:00   0   0   0   0   0   0   0   0   0  
2021-01-04 00:00:00+00:00   0   0   0   0   0   0   0   0   0  
2021-01-05 00:00:00+00:00   0   0   0   0   0   0   0   0   0  
...                        ..  ..  ..  ..  ..  ..  ..  ..  ..  
2025-01-02 00:00:00+00:00  10  10  10  10  10  10  10  10  10  
2025-01-03 00:00:00+00:00  10  10  10  10  10  10  10  10  10  
2025-01-04 00:00:00+00:00  10  10  10  10  10  10  10  10  10  
2025-01-05 00:00:00+00:00  10  10  10  10  10  10  10  10  10  
2025-01-06 00:00:00+00:00  10  10  10  10  10  10  10  10  10  

[1467 rows x 1728 columns]

In [24]:
def repeat_series_horizontally(series, target_shape):
    data = series.values.reshape(-1, 1)  # Convert to column vector
    return np.repeat(data, target_shape[1], axis=1)

nedded_copies = len(df_indicator.columns)
print('needed columns: ',nedded_copies)
print('needed rows: ',len(df_indicator))
print('shape:', df_indicator.shape)
print()

# copying the close price to the shape of the indicator
print('original shape:\n\n', df['close'])
dublicate_close_arr = repeat_series_horizontally(df['close'], df_indicator.shape)

print('\ndublicated dataframe as array:\n\n',dublicate_close_arr)

#turn the dublicate_close to a dataframe to keep timestamp index
df_dublicate_close  = pd.DataFrame(dublicate_close_arr,df.index)
df_dublicate_close 




needed columns:  1728
needed rows:  1467
shape: (1467, 1728)

original shape:

 timestamp
2021-01-01 00:00:00+00:00    29333.2
2021-01-02 00:00:00+00:00    32185.4
2021-01-03 00:00:00+00:00    33000.4
2021-01-04 00:00:00+00:00    31993.0
2021-01-05 00:00:00+00:00    33938.6
                              ...   
2025-01-02 00:00:00+00:00    96983.8
2025-01-03 00:00:00+00:00    98173.9
2025-01-04 00:00:00+00:00    98215.4
2025-01-05 00:00:00+00:00    98360.8
2025-01-06 00:00:00+00:00    99311.5
Name: close, Length: 1467, dtype: float64

dublicated dataframe as array:

 [[29333.2 29333.2 29333.2 ... 29333.2 29333.2 29333.2]
 [32185.4 32185.4 32185.4 ... 32185.4 32185.4 32185.4]
 [33000.4 33000.4 33000.4 ... 33000.4 33000.4 33000.4]
 ...
 [98215.4 98215.4 98215.4 ... 98215.4 98215.4 98215.4]
 [98360.8 98360.8 98360.8 ... 98360.8 98360.8 98360.8]
 [99311.5 99311.5 99311.5 ... 99311.5 99311.5 99311.5]]


,0,1,2,3,4,5,6,7,8,9,...,1718,1719,1720,1721,1722,1723,1724,1725,1726,1727
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-01-01 00:00:00+00:00,29333.2,29333.2,29333.2,29333.2,29333.2,29333.2,29333.2,29333.2,29333.2,29333.2,...,29333.2,29333.2,29333.2,29333.2,29333.2,29333.2,29333.2,29333.2,29333.2,29333.2
2021-01-02 00:00:00+00:00,32185.4,32185.4,32185.4,32185.4,32185.4,32185.4,32185.4,32185.4,32185.4,32185.4,...,32185.4,32185.4,32185.4,32185.4,32185.4,32185.4,32185.4,32185.4,32185.4,32185.4
2021-01-03 00:00:00+00:00,33000.4,33000.4,33000.4,33000.4,33000.4,33000.4,33000.4,33000.4,33000.4,33000.4,...,33000.4,33000.4,33000.4,33000.4,33000.4,33000.4,33000.4,33000.4,33000.4,33000.4
2021-01-04 00:00:00+00:00,31993.0,31993.0,31993.0,31993.0,31993.0,31993.0,31993.0,31993.0,31993.0,31993.0,...,31993.0,31993.0,31993.0,31993.0,31993.0,31993.0,31993.0,31993.0,31993.0,31993.0
2021-01-05 00:00:00+00:00,33938.6,33938.6,33938.6,33938.6,33938.6,33938.6,33938.6,33938.6,33938.6,33938.6,...,33938.6,33938.6,33938.6,33938.6,33938.6,33938.6,33938.6,33938.6,33938.6,33938.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-02 00:00:00+00:00,96983.8,96983.8,96983.8,96983.8,96983.8,96983.8,96983.8,96983.8,96983.8,96983.8,...,96983.8,96983.8,96983.8,96983.8,96983.8,96983.8,96983.8,96983.8,96983.8,96983.8
2025-01-03 00:00:00+00:00,98173.9,98173.9,98173.9,98173.9,98173.9,98173.9,98173.9,98173.9,98173.9,98173.9,...,98173.9,98173.9,98173.9,98173.9,98173.9,98173.9,98173.9,98173.9,98173.9,98173.9
2025-01-04 00:00:00+00:00,98215.4,98215.4,98215.4,98215.4,98215.4,98215.4,98215.4,98215.4,98215.4,98215.4,...,98215.4,98215.4,98215.4,98215.4,98215.4,98215.4,98215.4,98215.4,98215.4,98215.4


## compare the shape of the copied closing prices 

In [25]:

# Create a DataFrame with the same shape as the indicator
df_custom_indiator = df_indicator

if df_custom_indiator.shape == df_dublicate_close.shape:
    df_renamed_close = df_dublicate_close
    df_renamed_close.columns = df_custom_indiator.columns
    print("Columns have been renamed ")
else:
    raise ValueError("The DataFrames do not have the same shape.")

# Display the resulting DataFrame

print(f'Signal df shape: {df_indicator.shape} \nRenamed df shape: {df_renamed_close.shape}')
df_renamed_close



Columns have been renamed 
Signal df shape: (1467, 1728) 
Renamed df shape: (1467, 1728)


momentum_atr_length              5                                      \
momentum_ema_length            100                                       
momentum_ema2_length            10                                       
momentum_vola_multiplier       1.6                        1.7            
momentum_vol_window              7        8        9        7        8   
timestamp                                                                
2021-01-01 00:00:00+00:00  29333.2  29333.2  29333.2  29333.2  29333.2   
2021-01-02 00:00:00+00:00  32185.4  32185.4  32185.4  32185.4  32185.4   
2021-01-03 00:00:00+00:00  33000.4  33000.4  33000.4  33000.4  33000.4   
2021-01-04 00:00:00+00:00  31993.0  31993.0  31993.0  31993.0  31993.0   
2021-01-05 00:00:00+00:00  33938.6  33938.6  33938.6  33938.6  33938.6   
...                            ...      ...      ...      ...      ...   
2025-01-02 00:00:00+00:00  96983.8  96983.8  96983.8  96983.8  96983.8   
2025-01-03 00:00:00+00:00  98173.9  98173.9  98173.9  98173.9  98173.9   
2025-01-04 00:00:00+00:00  98215.4  98215.4  98215.4  98215.4  98215.4   
2025-01-05 00:00:00+00:00  98360.8  98360.8  98360.8  98360.8  98360.8   
2025-01-06 00:00:00+00:00  99311.5  99311.5  99311.5  99311.5  99311.5   

momentum_atr_length                                                     ...  \
momentum_ema_length                                                     ...   
momentum_ema2_length                                                15  ...   
momentum_vola_multiplier                1.8                        1.6  ...   
momentum_vol_window              9        7        8        9        7  ...   
timestamp                                                               ...   
2021-01-01 00:00:00+00:00  29333.2  29333.2  29333.2  29333.2  29333.2  ...   
2021-01-02 00:00:00+00:00  32185.4  32185.4  32185.4  32185.4  32185.4  ...   
2021-01-03 00:00:00+00:00  33000.4  33000.4  33000.4  33000.4  33000.4  ...   
2021-01-04 00:00:00+00:00  31993.0  31993.0  31993.0  31993.0  31993.0  ...   
2021-01-05 00:00:00+00:00  33938.6  33938.6  33938.6  33938.6  33938.6  ...   
...                            ...      ...      ...      ...      ...  ...   
2025-01-02 00:00:00+00:00  96983.8  96983.8  96983.8  96983.8  96983.8  ...   
2025-01-03 00:00:00+00:00  98173.9  98173.9  98173.9  98173.9  98173.9  ...   
2025-01-04 00:00:00+00:00  98215.4  98215.4  98215.4  98215.4  98215.4  ...   
2025-01-05 00:00:00+00:00  98360.8  98360.8  98360.8  98360.8  98360.8  ...   
2025-01-06 00:00:00+00:00  99311.5  99311.5  99311.5  99311.5  99311.5  ...   

momentum_atr_length              7                                      \
momentum_ema_length            240                                       
momentum_ema2_length            40       45                              
momentum_vola_multiplier       1.8      1.6                        1.7   
momentum_vol_window              9        7        8        9        7   
timestamp                                                                
2021-01-01 00:00:00+00:00  29333.2  29333.2  29333.2  29333.2  29333.2   
2021-01-02 00:00:00+00:00  32185.4  32185.4  32185.4  32185.4  32185.4   
2021-01-03 00:00:00+00:00  33000.4  33000.4  33000.4  33000.4  33000.4   
2021-01-04 00:00:00+00:00  31993.0  31993.0  31993.0  31993.0  31993.0   
2021-01-05 00:00:00+00:00  33938.6  33938.6  33938.6  33938.6  33938.6   
...                            ...      ...      ...      ...      ...   
2025-01-02 00:00:00+00:00  96983.8  96983.8  96983.8  96983.8  96983.8   
2025-01-03 00:00:00+00:00  98173.9  98173.9  98173.9  98173.9  98173.9   
2025-01-04 00:00:00+00:00  98215.4  98215.4  98215.4  98215.4  98215.4   
2025-01-05 00:00:00+00:00  98360.8  98360.8  98360.8  98360.8  98360.8   
2025-01-06 00:00:00+00:00  99311.5  99311.5  99311.5  99311.5  99311.5   

momentum_atr_length                                                     
momentum_ema_length                                              

In [62]:
# Numba-compiled order function
@njit
def order_func_nb(c, high, low, open_, entries, sl_prices, tp_prices,entry_price,atr_values):
    close_price = c.close[c.i, c.col]
    # print("INDEX", c.i)
    # print("COL", c.col)
    # print("high", high[c.i])
    # print('open', open_[c.i])
    # print("close", close_price)
    # print('low', low[c.i])
    # print('position size :', c.position_now)
    # print('cash:', c.cash_now)
    # print('entries:', entries[c.i, c.col])
    # print('sl_prices:', sl_prices[c.i])
    # print('tp_prices:', tp_prices[c.i])
    # print('entry_price:', entry_price[c.i])
    # print()

# if in position 
    if (c.position_now > 0):
        # Check if SL is hit
        # if c.close[c.i-1, c.col] <= sl_prices[c.i]:
        if open_[c.i] <= sl_prices[c.i]:
        # if low[c.i] <= sl_prices[c.i]:
            value = vbt.portfolio.nb.order_nb(
                size=-np.inf,  # Close position
                price=open_[c.i],  # Current closing price
                size_type=SizeType.Amount,
                direction=Direction.LongOnly,
                fees=0.001,
                slippage=0.002)
            # print('sl hit at index',c.i)
            # print('sl order', value)

            return value

        # sl update for long position


        if (entries[c.i-1, c.col] == 5):  
            highest_low = np.max(low[c.i-7:c.i])
            update2 = highest_low - atr_values[c.i] * 0.2
            if update2 > sl_prices[c.i]:
                # print('sl update before',c.i,sl_prices[c.i])
                sl_prices[:]= update2

        #losen sl for less valatility 
        if (entries[c.i-1, c.col] == 10): 
            highest_low = np.max(low[c.i-7:c.i])
            update = highest_low - atr_values[c.i]
            if update > sl_prices[c.i]:
                # print('sl update before',c.i,sl_prices[c.i])
                sl_prices[:]= update
  


    # if not in position search for position to enter
    elif (c.position_now == 0) & (c.i != 0):
        if entries[c.i-1, c.col] == 10:
            sl_prices[:] = np.nan
            entry_price[:] = open_[c.i]
            order = vbt.portfolio.nb.order_nb(
                size=1,  # Adjusted order size
                price=open_[c.i],  # Current closing price
                size_type=SizeType.Percent,  # Specify size type
                direction=Direction.LongOnly,  # Long-only trading
                fees=0.001,  # No fees
                slippage=0.002,  # No slippage
                allow_partial=False,  # Do not allow partial fills
                raise_reject=True  # Raise an error if the order is rejected
            )

            sl_prices[:] = low[c.i] - atr_values[c.i] 
            # print('entry', order)
            return order

    
    return vbt.portfolio.enums.NoOrder


close = df_renamed_close
entries = df_custom_indiator.to_numpy()

open_ = df['open'].to_numpy().flatten()
high = df['high'].to_numpy().flatten()
low = df['low'].to_numpy().flatten()

atr_values = talib.ATR(df['high'], df['low'], df['close'], timeperiod=7).to_numpy().flatten()

# Create an array to store SL prices
sl_prices = np.full(close.shape[0], np.nan)  # Use a 1D array
tp_prices = np.full(close.shape[0], np.nan)  # Use a 1D array
entry_price = np.full(close.shape[0], np.nan)  # Use a 1D array

# Create portfolio
pf = vbt.Portfolio.from_order_func(
    close,           # Price DataFrame
    order_func_nb,
    high,
    low,
    open_,
    entries,    # Order function
    sl_prices,
    tp_prices,
    entry_price,
    atr_values,  # Pass the SL prices array
    init_cash=500  # Initial cash balance
)

# Display some portfolio performance metrics
print("Total Return:", pf.total_return())
print("\nOrder Records:")

pf.orders.records_readable




Total Return: momentum_atr_length  momentum_ema_length  momentum_ema2_lengh  momentum_vola_multiplier  momentum_vol_window
5                    100                  10                   1.6                       7                      0.658777
                                                                                         8                      0.951434
                                                                                         9                      0.684537
                                                               1.7                       7                      0.627589
                                                                                         8                      1.143886
                                                                                                                  ...   
7                    240                  45                   1.7                       8                      0.733480
                              

,Order Id,Column,Timestamp,Size,Price,Fees,Side
0,0,"(5, 100, 10, 1.6, 7)",2021-04-11 00:00:00+00:00,0.008340,59889.7404,0.499500,Buy
1,1,"(5, 100, 10, 1.6, 7)",2021-04-18 00:00:00+00:00,0.008340,59875.3094,0.499380,Sell
2,2,"(5, 100, 10, 1.6, 7)",2021-04-28 00:00:00+00:00,0.009041,55124.2284,0.498382,Buy
3,3,"(5, 100, 10, 1.6, 7)",2021-05-05 00:00:00+00:00,0.009041,53089.9074,0.479990,Sell
4,4,"(5, 100, 10, 1.6, 7)",2021-05-07 00:00:00+00:00,0.008477,56510.4954,0.479031,Buy
...,...,...,...,...,...,...,...
176299,176299,"(7, 240, 45, 1.8, 9)",2024-12-12 00:00:00+00:00,0.008378,101319.1338,0.848832,Buy
176300,176300,"(7, 240, 45, 1.8, 9)",2024-12-19 00:00:00+00:00,0.008378,100002.4942,0.837802,Sell
176301,176301,"(7, 240, 45, 1.8, 9)",2024-12-29 00:00:00+00:00,0.008756,95495.6100,0.836128,Buy
176302,176302,"(7, 240, 45, 1.8, 9)",2024-12-31 00:00:00+00:00,0.008756,92606.4160,0.810831,Sell


In [63]:
total_return = pf.total_return()
max_dd = pf.max_drawdown()
return_and_maxdd = pd.concat([total_return, max_dd], axis=1)
return_and_maxdd.sort_values(by='total_return', ascending=False).head(50)



total_return  \
momentum_atr_length momentum_ema_length momentum_ema2_lengh momentum_vola_multiplier momentum_vol_window                 
7                   160                 25                  1.6                      8                        2.015724   
                                        10                  1.6                      8                        2.015724   
                                        15                  1.6                      8                        2.015724   
                                        35                  1.6                      8                        2.015724   
                                        40                  1.6                      8                        2.015724   
                                        30                  1.6                      8                        2.015724   
                                        45                  1.6                      8                        2.015724   
                                        20                  1.6                      8                        2.015724   
5                   160                 35                  1.7                      8                        1.972393   
                                        30                  1.7                      8                        1.972393   
                                        45                  1.7                      8                        1.972393   
                                        25                  1.7                      8                        1.972393   
                                        20                  1.7                      8                        1.972393   
                                        15                  1.7                      8                        1.972393   
                                        10                  1.7                      8                        1.972393   
                                        40                  1.7                      8                        1.972393   
6                   160                 15                  1.6                      8                        1.941557   
                                        40                  1.6                      8                        1.941557   
                                        35                  1.6                      8                        1.941557   
                                        45                  1.6                      8                        1.941557   
                                        30                  1.6                      8                        1.941557   
                                        20                  1.6                      8                        1.941557   
                                        25                  1.6                      8                        1.941557   
                                        10                  1.6                      8                        1.941557   
                                        45                  1.7                      8                        1.858552   
                                        40                  1.7                      8                        1.858552   
                                        10                  1.7                      8                        1.858552   
                                        20                  1.7                      8                        1.858552   
                                        15                  1.7                      8                        1.858552   
                                        35                  1.7                      8                        1.858552   
                                        30                  1.7                      8                        1.858552   
                                        25                  1.7                 

In [64]:
# Access the setup for a specific name, e.g., '(4, 100, 1.0, 4)'
setup_name = (8, 160, 1.8, 7)
specific_pf = pf[setup_name]
specific_pf.plot().show()



KeyError: (8, 160, 1.8, 7)

In [36]:
specific_pf2 = pf.select_one(setup_name)
orders = specific_pf2.orders.records_readable
orders

,Order Id,Column,Timestamp,Size,Price,Fees,Side
0,105,"(8, 160, 1.8, 7)",2021-07-31 00:00:00+00:00,0.011812,42287.3058,0.499500,Buy
1,106,"(8, 160, 1.8, 7)",2021-08-04 00:00:00+00:00,0.011812,38125.1968,0.450337,Sell
2,107,"(8, 160, 1.8, 7)",2021-08-07 00:00:00+00:00,0.010470,42925.1790,0.449438,Buy
3,108,"(8, 160, 1.8, 7)",2021-08-31 00:00:00+00:00,0.010470,46887.0380,0.490919,Sell
4,109,"(8, 160, 1.8, 7)",2021-09-01 00:00:00+00:00,0.010382,47189.6910,0.489938,Buy
...,...,...,...,...,...,...,...
92,197,"(8, 160, 1.8, 7)",2024-12-12 00:00:00+00:00,0.013208,101319.1338,1.338199,Buy
93,198,"(8, 160, 1.8, 7)",2024-12-19 00:00:00+00:00,0.013208,100002.4942,1.320809,Sell
94,199,"(8, 160, 1.8, 7)",2024-12-26 00:00:00+00:00,0.013230,99633.7698,1.318170,Buy
95,200,"(8, 160, 1.8, 7)",2024-12-30 00:00:00+00:00,0.013230,93552.3204,1.237712,Sell


In [39]:
orders_time_filered = orders[orders['Timestamp'] > '2023-01-01 00:00:00']
orders_time_filered.tail(50)

,Order Id,Column,Timestamp,Size,Price,Fees,Side
47,152,"(8, 160, 1.8, 7)",2023-10-12 00:00:00+00:00,0.020811,26824.9426,0.558250,Sell
48,153,"(8, 160, 1.8, 7)",2023-10-22 00:00:00+00:00,0.018591,29967.4152,0.557135,Buy
49,154,"(8, 160, 1.8, 7)",2023-11-15 00:00:00+00:00,0.018591,35480.2972,0.659626,Sell
50,155,"(8, 160, 1.8, 7)",2023-11-17 00:00:00+00:00,0.018167,36236.5284,0.658308,Buy
51,156,"(8, 160, 1.8, 7)",2023-12-12 00:00:00+00:00,0.018167,41166.5020,0.747871,Sell
52,157,"(8, 160, 1.8, 7)",2023-12-16 00:00:00+00:00,0.017761,42022.2768,0.746377,Buy
53,158,"(8, 160, 1.8, 7)",2023-12-18 00:00:00+00:00,0.017761,41291.5514,0.733398,Sell
54,159,"(8, 160, 1.8, 7)",2023-12-19 00:00:00+00:00,0.017125,42741.2118,0.731933,Buy
55,160,"(8, 160, 1.8, 7)",2023-12-29 00:00:00+00:00,0.017125,42478.9718,0.727442,Sell
56,161,"(8, 160, 1.8, 7)",2023-12-30 00:00:00+00:00,0.017223,42152.8374,0.725989,Buy
